# Mixed effect model analysis

## Load data and packages

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import statsmodels.formula.api as smf

In [2]:
df = pd.read_excel("./data/df_for_analysis.xlsx",index_col=0)

In [3]:
def keep_weight_post_infection(x,weight_end = "weight_T13"):
    """
    Given a DataFrame `x`, returns a Series containing the weights post-infection.

    *Arguments*
    - x: DataFrame containing the data.
    - weight_end: Column name of the last weight measurement.

    *Returns*
    - shifted_series: Series containing the weights post-infection.
    """
    # Extract relevant columns from the DataFrame
    dates = x['Dates']
    t_infection = x['Time_infection']
    datas = x['weight_T_infection':weight_end]

    # Find the date closest to the infection time
    new_time_infection = dates[dates <= t_infection][-1]
    location_of_TI = dates.get_loc(new_time_infection)

    # Return the original series if the data at the infection time is NaN
    if np.isnan(datas[location_of_TI]):
        return datas
    
    # Shift the values of the input series by the specified index
    shifted_series = pd.Series([np.nan] * len(datas), index=datas.index)
    if location_of_TI == 0:
        return datas
    else:
        shifted_series[:-location_of_TI] = datas.values.tolist()[location_of_TI:]
    # Shift the values of the input series by the specified index
    return shifted_series

### Transform data to long format

Keep only data that are after the time of infeciton

In [4]:
# change dates column to datetimindex and transform weight datas to numeric only
df.loc[:,"weight_T_infection":"weight_T13"] = df.loc[:,"weight_T_infection":"weight_T13"].apply(pd.to_numeric,errors='coerce')
serie_dates = df['Time_point'].apply(lambda x: pd.to_datetime(x.split(','),dayfirst=True))
df['Dates'] = serie_dates
data = df.apply(lambda x: keep_weight_post_infection(x),axis=1)
data

C:\Users\ma7054\AppData\Local\Temp\ipykernel_9032\2938565056.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.loc[:,"weight_T_infection":"weight_T13"] = df.loc[:,"weight_T_infection":"weight_T13"].apply(pd.to_numeric,errors='coerce')


,weight_T_infection,weight_T1,weight_T2,weight_T3,weight_T4,weight_T5,weight_T6,weight_T7,weight_T8,weight_T9,weight_T10,weight_T11,weight_T12,weight_T13
0,23.92,21.72,20.96,19.38,18.16,16.44,15.49,15.44,15.05,NaN,NaN,NaN,NaN,NaN
1,21.40,19.45,18.84,17.82,16.80,15.02,14.14,14.40,14.73,NaN,NaN,NaN,NaN,NaN
2,22.56,21.45,20.83,18.67,16.82,15.30,14.74,14.80,14.42,NaN,NaN,NaN,NaN,NaN
3,20.39,18.69,16.60,15.58,14.17,13.54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23.72,21.74,20.29,19.56,18.50,16.80,16.78,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,27.70,25.00,22.80,21.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2353,26.10,24.90,22.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2354,24.60,25.80,25.70,24.90,22.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2355,27.40,27.30,28.20,27.80,27.70,27.80,27.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Normalization by weight at T infection and replace the normalize data into the original dataframe

In [5]:

normalize = data.div(data['weight_T_infection'],axis=0)*100
df_normalize = df.copy()
df_normalize.loc[:,"weight_T_infection":"weight_T13"] = normalize

df_normalize['min_weight'] = df_normalize.loc[:,"weight_T_infection":"weight_T13"].min(axis=1)
df_normalize['t_origin'] = df_normalize['time_original']
df_normalize

,Mouse_ID,ID_Experiment,Cage,Strain,Date,Experiment,Group,Group_info,H0,Pre_traitment,...,survival_0.06,time_0.05,survival_0.05,time_original,survival_original,max_loss_weight_percentage,exp,sub_exp,min_weight,t_origin
0,TRO-05432,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.629181,1,A,62.918060,9.0
1,TRO-05433,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,9.0,1,0.660748,1,A,66.074766,9.0
2,TRO-05434,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,2.5,1,9.0,1,0.639184,1,A,63.918440,9.0
3,TRO-05435,ID_001,A,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,6.0,1,0.664051,1,A,66.405101,6.0
4,TRO-05456,ID_001,B,BALB/cByJ,2014-06-05,Candida/Propionate,1A,Propionate / 2*10^5,1,propionate,...,1,1.5,1,7.0,1,0.707420,1,A,70.741990,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,TRO-028337,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,1.5,1,5.0,1,0.761733,3,no,76.173285,5.0
2353,TRO-028338,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,2.5,1,4.0,1,0.865900,3,no,86.590038,4.0
2354,TRO-028339,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,1,5.5,1,6.0,1,0.926829,3,no,92.682927,6.0
2355,TRO-028342,ID_096,ETRO-01911,C57BL/6J,2023-03-03,Pneumococcus/Training/Cross-fostering/male,3,D. Zy-Zy,1,training/cross-fostering,...,0,8.0,0,8.0,0,0.996350,3,no,99.635036,8.0


Find only releavent columns


In [6]:
columns = df_normalize.loc[:,"weight_T_infection":"weight_T13"].columns.tolist()
columns_index = df_normalize.loc[:, ~df_normalize.columns.isin(columns)]
column_time = [n for n in df_normalize.columns.tolist() if "time_" in n]
column_time = [column_time[-1]] + column_time[:-1]
columns_index_time = ['ID_Experiment','Mouse_ID','Date','Infection','Group','exp','survival_original','t_origin']

Transform to tidy data for releavent columns

In [7]:
df_longer_weight = df_normalize.melt(id_vars=columns_index_time,value_vars=columns,var_name="Time",value_name="weight")
df_longer_weight['Time'] = df_longer_weight['Time'].apply(lambda x: "".join(x.split("_")[1:]))
df_longer_weight

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
32993,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T13,NaN
32994,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T13,NaN
32995,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T13,NaN
32996,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,8.0,T13,NaN


In [8]:
df_dead = df_longer_weight[df_longer_weight['survival_original']==1]
df_survive = df_longer_weight[df_longer_weight['survival_original']==0]
df_dead_T6 = df_dead[df_dead['Time'] == 'T6']

In [9]:
stats = df_dead_T6.groupby(['Infection'])['weight'].agg(['mean','median','count','std'])
stats['ci95_hi'] = stats['median'] + 1.96*stats['std']/np.sqrt(stats['count'])
stats['ci95_low'] = stats['median'] - 1.96*stats['std']/np.sqrt(stats['count'])
stats


,mean,median,count,std,ci95_hi,ci95_low
Infection,,,,,,
C. albicans,72.756838,70.700637,51,7.882190,72.863943,68.537331
H1N1,74.404462,72.855544,66,6.899504,74.520114,71.190974
Listeria,74.637131,74.015748,7,3.225896,76.405525,71.625971
S. pneumoniae,91.887905,91.666667,57,8.133820,93.778274,89.555059


In [10]:
df_survive.groupby(["Infection",'Time'])['weight'].agg(['mean',"median","count","std"])

mean      median  count        std
Infection     Time                                                
C. albicans   T1           93.423282   94.154432    164   3.919565
              T10          90.400010   92.140206    102   9.440268
              T11          90.242063   92.183309    102  10.286005
              T12          90.102767   93.007511    100  11.339066
              T13          91.540737   94.016363     36   8.440398
              T2           88.427936   89.560698    164   5.749952
              T3           86.554831   86.577325    164   7.968843
              T4           88.285082   87.634409    164   9.080601
              T5           90.033289   90.886018    164   9.681441
              T6           90.578154   92.713018    102   9.287597
              T7           89.577994   91.196728    102   8.403846
              T8           90.142214   91.973951    102   9.040421
              T9           88.228944   90.148129    102   9.108362
              Tinfection  100.000000  100.000000    164   0.000000
H1N1          T1           98.799998   99.450549    203   4.389058
              T10          92.525746   96.423849    110  11.877953
              T11          92.458431  102.150538     25  14.992576
              T12          83.984279   81.724263     14  13.535254
              T13                NaN         NaN      0        NaN
              T2           96.113158   96.250000    203   5.879753
              T3           94.211246   95.667870    203   7.771809
              T4           92.398960   92.893401    203   9.243240
              T5           90.654343   91.484071    186  10.364847
              T6           89.390903   91.489362    177  11.622416
              T7           90.605853   93.816239    170  11.694513
              T8           92.888478   96.461727    146  11.791498
              T9           92.388209   96.634779    138  11.797687
              Tinfection  100.000000  100.000000    203   0.000000
Listeria      T1           98.757687   98.963731    555   3.541511
              T10         102.765257  102.544006     16   4.244996
              T11         103.749123  102.834704     16   4.050310
              T12                NaN         NaN      0        NaN
              T13                NaN         NaN      0        NaN
              T2           92.697426   92.610837    552   5.679112
              T3           91.146573   91.904762    495   8.047336
              T4           92.217347   93.750000    415   8.854193
              T5           93.906125   95.186615    290   8.890645
              T6           95.533197   96.954315    211   8.724870
              T7           95.709996   96.634615    155   7.958578
              T8           96.455673   97.396208     98   8.140584
              T9           95.848173   95.831754     40  10.562874
              Tinfection  100.000000  100.000000    555   0.000000
S. pneumoniae T1           97.183700   97.560976    325   5.940770
              T10          98.865805  100.526316     75  11.332548
              T11         100.135090  102.204637     68  12.040936
              T12          98.831040   99.130435      7   6.552113
              T13                NaN         NaN      0        NaN
              T2           98.288156   98.916911    328   6.178529
              T3           98.663176   99.607239    328   6.575897
              T4           97.964765   99.007291    308   7.159976
              T5           98.937685  100.000000    287   7.179290
              T6           99.743048  101.081081    245   7.499001
              T7          100.563740  101.395470    224   7.554573
              T8          100.016669  101.428571    183   8.841656
              T9          100.136391  101.435703    140   9.183938
              Tinfection  100.000000  100.000000    328   0.000000

save the datas

In [11]:
stats.groupby(['Infection']).min()

,mean,median,count,std,ci95_hi,ci95_low
Infection,,,,,,
C. albicans,72.756838,70.700637,51,7.882190,72.863943,68.537331
H1N1,74.404462,72.855544,66,6.899504,74.520114,71.190974
Listeria,74.637131,74.015748,7,3.225896,76.405525,71.625971
S. pneumoniae,91.887905,91.666667,57,8.133820,93.778274,89.555059


In [12]:
df_longer_weight.to_excel("./data/df_long_format_for_analysis.xlsx")

## Mixed effect model
### function

In [13]:
def Mixed_Effects_Models(df,chosen_infection = 'S. pneumoniae',time_to_exclude = 8):
    df_infection = df[df['Infection'] == chosen_infection]
    df_infection = df_infection[~df_infection['Time'].isin([f"T{n}" for n in range(time_to_exclude,15,1)])]#remove unused data
    
    time_point = df_infection['Time'].unique()
    weight_point_to_integer = dict(zip(time_point,[n for n in range(len(time_point))]))
    
    df_infection['Time'] = df_infection['Time'].replace(weight_point_to_integer)
    model = smf.mixedlm("weight ~ Time",df_infection,groups=df_infection['survival_original'],missing="drop").fit()
    return model.summary()

In [14]:
Mixed_Effects_Models(df_longer_weight,"S. pneumoniae",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 2118    Method:             REML      
No. Groups:       2       Scale:              24.4583   
Min. group size:  981     Log-Likelihood:     -6394.4628
Max. group size:  1137    Converged:          Yes       
Mean group size:  1059.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   99.816    0.922 108.310 0.000 98.010 101.622
Time        -2.233    0.131 -17.003 0.000 -2.491  -1.976
Group Var    1.642    0.478                             
========================================================

"""

In [15]:
Mixed_Effects_Models(df_longer_weight,"Listeria",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 3104    Method:             REML      
No. Groups:       2       Scale:              13.6527   
Min. group size:  1442    Log-Likelihood:     -8466.5944
Max. group size:  1662    Converged:          Yes       
Mean group size:  1552.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.706    1.333  75.551 0.000 98.094 103.319
Time        -4.890    0.081 -60.147 0.000 -5.049  -4.731
Group Var    3.532    1.347                             
========================================================

"""

In [16]:
Mixed_Effects_Models(df_longer_weight,"C. albicans",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1199    Method:             REML      
No. Groups:       2       Scale:              38.4846   
Min. group size:  379     Log-Likelihood:     -3892.7082
Max. group size:  820     Converged:          Yes       
Mean group size:  599.5                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   97.468    1.627  59.907 0.000 94.279 100.657
Time        -3.865    0.127 -30.525 0.000 -4.113  -3.617
Group Var    5.102    1.181                             
========================================================

"""

In [17]:
Mixed_Effects_Models(df_longer_weight,"H1N1",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1666    Method:             REML      
No. Groups:       2       Scale:              35.8096   
Min. group size:  651     Log-Likelihood:     -5348.6435
Max. group size:  1015    Converged:          Yes       
Mean group size:  833.0                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.074    2.232  44.837 0.000 95.699 104.448
Time        -2.995    0.104 -28.759 0.000 -3.199  -2.790
Group Var    9.833    2.335                             
========================================================

"""